In [93]:
# pip install pyodbc

In [2]:
import pandas as pd
import pyodbc
import os

In [8]:
# Define the parameters for connect to SQL Server database
driver_name = "ODBC Driver 17 for SQL Server"
server_name = 'TIT-20230930FXD\\SQLEXPRESS'
database_name = 'brazilian_ecommerce' # database_name = 'master'
trusted_connection = 'yes'

# Define connection string
conn_str = f'DRIVER={driver_name};SERVER={server_name};DATABASE={database_name};Trusted_Connection={trusted_connection}'

In [9]:
# อ่านข้อมูลจากไฟล์ CSV โดยใช้ pandas
df = pd.read_csv(r'data\\olist_geolocation_dataset.csv')
df.dtypes

geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object

In [10]:
df.columns

Index(['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'geolocation_city', 'geolocation_state'],
      dtype='object')

In [11]:
df["geolocation_zip_code_prefix"] = df["geolocation_zip_code_prefix"].astype(object)

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 738332 entries, 0 to 1000161
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   geolocation_zip_code_prefix  738332 non-null  object 
 1   geolocation_lat              738332 non-null  float64
 2   geolocation_lng              738332 non-null  float64
 3   geolocation_city             738332 non-null  object 
 4   geolocation_state            738332 non-null  object 
dtypes: float64(2), object(3)
memory usage: 33.8+ MB


In [14]:
# sql_cmd = """
#     CREATE TABLE geolocation(    
#         geolocation_zip_code_prefix VARCHAR(255),
#         geolocation_lat VARCHAR(255),
#         geolocation_lng VARCHAR(255),
#         geolocation_city VARCHAR(255),
#         geolocation_state VARCHAR(255)
#     );
# """

# with pyodbc.connect(conn_str) as con:
#     con.execute(sql_cmd)

In [15]:
# กำหนดคำสั่ง SQL เพื่อ truncate ตาราง
truncate_query = '''
    TRUNCATE TABLE geolocation
'''

# Execute คำสั่ง SQL เพื่อ truncate ตาราง
with pyodbc.connect(conn_str).cursor() as cursor:
    cursor.execute(truncate_query)
    
pyodbc.connect(conn_str).commit()

In [17]:
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    for index, row in df.iterrows():
        insert_query = """
            INSERT INTO geolocation
            VALUES (?, ?, ?, ?, ?)
        """
        # Execute คำสั่ง SQL เพื่อเพิ่มข้อมูล
        cursor.execute(insert_query, tuple(row))
    con.commit()

In [18]:
with pyodbc.connect(conn_str) as con:
    df_query = pd.read_sql("SELECT * FROM geolocation", con)
    
df_query.info()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6740\2709571292.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql("SELECT * FROM geolocation", conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738332 entries, 0 to 738331
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   geolocation_zip_code_prefix  738332 non-null  object
 1   geolocation_lat              738332 non-null  object
 2   geolocation_lng              738332 non-null  object
 3   geolocation_city             738332 non-null  object
 4   geolocation_state            738332 non-null  object
dtypes: object(5)
memory usage: 28.2+ MB
